In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI

client = OpenAI(api_key='')

In [ ]:
def topic(sentence):
  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
        {"role": "system",
         "content": "Identify the main idea of the following sentences. Only give me the sentences of the main idea."},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5
  )

  return completion.choices[0].message.content

In [ ]:
def removal(sentence):
  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
        {"role": "system",
         "content": "Remove any information that is repetitive or not crucial to understanding the main idea and the supporting points."},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5
  )

  return completion.choices[0].message.content

In [ ]:
def remove_details(sentence):
  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
        {"role": "system",
         "content": "Remove all details such as reasons, facts, examples, or explanations."},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5
  )

  return completion.choices[0].message.content

In [ ]:
def simple_sent(sentence):
  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
        {"role": "system",
         "content": "Make this sentence simpler without changing sentence meaning."},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5
  )

  return completion.choices[0].message.content

In [ ]:
def simplify(statement):
    completion = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": f"Simplify the following sentences: {statement}",
            }
        ],
        temperature=0.5,
        top_p=0.5,
    )
    return completion.choices[0].message.content

def check_freq(sentence):

  nlp = spacy.load("en_core_web_sm")
  words = [tok.lemma_ for tok in nlp(sentence) if tok.pos_ not in ["PUNCT", "SPACE"]]

  freq_dict = {}
  for word in words:
    freq = word_frequency(word, 'en')
    freq_dict[word] = freq

  vocab = dict(sorted(freq_dict.items(), key=lambda item: item[1]))
  return vocab

def words_exp(sentence):

  freq_dict = check_freq(sentence)

  explain = dict((k, v) for k, v in freq_dict.items() if v < 1e-4)

  words = list(explain.keys())

  return words

def simplify_words(sentence):
  words = words_exp(sentence)
  assis = 'Give me a new verison of sentences which replace these words in simpler synonyms or explanations:'
  for word in words:
    assis += word
    assis += ', '
  assis += 'inside sentences and only give me the new version of explained sentences. Please combine with orginal sentence meanings and keep the original meanings.'

  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
         {"role": "system",
         "content": assis},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5,
  )

  return completion.choices[0].message.content

def simplify_structure(sentence):
  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
        {"role": "system",
         "content": "Simpler sentences' structure, do not give me several points but a coherent paragraph, and not changing original sentences' meanings."},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5,
  )
  return completion.choices[0].message.content

def output(sentence):
    tmp = simplify(sentence)
    tmp = simplify_structure(tmp)
    tmp = simplify_words(tmp)
    tmp = simplify_structure(tmp)
    tmp = simplify_words(tmp)
    return tmp

In [ ]:
def summary(sentence):
  tmp = topic(sentence)
  tmp2 = output(tmp)
  tmp3 = removal(tmp2)
  tmp4 = remove_details(tmp3)
  res = simple_sent(tmp4)
  return res

In [ ]:
summary("Managing down operating costs within such a complex technology environment is extremely difficult. Retiring applications that are tightly coupled within the technical and data architecture results in nearly insurmountable dependency management issues and change risk. Seemingly small projects quickly become big, expensive efforts with extended timeframes. Retiring one application inevitably requires upstream and downstream changes and this added cost erodes the business case and often leads to a decision to leave things as they are.")

"Removing old software can be hard and might cause problems with other parts. It can be expensive, so people often don't change the system."

In [ ]:
summary("A constant failure rate (CFR) is indicative of externally induced failures, as in the constant hazard rate situation for non-repairable items. A CFR is also typical of complex systems subject to repair and overhaul, where different parts exhibit different patterns of failure with time and parts have different ages since repair or replacement. Repairable systems can show a decreasing failure rate (DFR) when reliability is improved by progressive repair, as defective parts which fail relatively early are replaced by good parts. ‘Burn in’ is applied to electronic systems, as well as to parts, for this purpose.")

'Machines often stop working because of outside issues. Keeping them up can make them work better and more dependably.'

In [ ]:
summary("Neoclassical economics is built on the assumption that humans are rational beings who have a clear idea of their best interests and strive to extract maximum benefit (or 'utility', in economist-speak) from any situation.")

'Traditional economic theory says people make smart choices to get the most benefits.'

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
df = df.reset_index(drop=True)
original = df[df.columns[0]].tolist()
results = []
for statement in original:
    summarized = summary(statement)
    results.append(summarized)

df_combined = pd.DataFrame({
    'OriginalStatements': original,
    'SummarizationStatements': results,
})


df_combined.to_csv('summarization.csv', index=False)